In [3]:
import warnings
import sys
import os
import pathlib
import torch 
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import HTML

warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

SEED = 40
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import loaders.goes
import fields.vector_field

In [ ]:
date = "2024-09-18"
band = 8

# Load data
dsf = loaders.goes.discrete_scalar_field("2024-09-18", 
                                            band = 8, 
                                            start = "00:00", 
                                            end = "04:05", 
                                            by = "00:15", 
                                            extent=(-86.1, -75.1, 30, 37.1))

# Create nested output folder: date/levelmb
folder_name = os.path.join("goes", date, f"{band}")
os.makedirs(folder_name, exist_ok=True)

# Plot and save scalar field at start and end frames
for frame in [0,dsf.coord_field.times.shape[0]-1]:
    fig = dsf.plot(frame=frame)
    fig.savefig(os.path.join(folder_name, f"dsf_frame{frame}.png"))
    plt.close(fig)


 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC-M6C08_G16_s20242620001170_e20242620003543_c20242620004049.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC-M6C08_G16_s20242620031170_e20242620033543_c20242620034039.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC-M6C08_G16_s20242620021170_e20242620023543_c20242620024041.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC-M6C08_G16_s20242620016170_e20242620018543_c20242620019029.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC-M6C08_G16_s20242620026170_e20242620028543_c20242620029039.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/262/00/OR_ABI-L2-CMIPC

In [21]:
dsf.coord_field.times / 3600

tensor([[0.0433],
        [0.2933],
        [0.5433],
        [0.7933],
        [1.0433],
        [1.2933],
        [1.5433],
        [1.7933],
        [2.0433],
        [2.2933],
        [2.5433],
        [2.7933],
        [3.0433],
        [3.2933],
        [3.5433],
        [3.7933],
        [4.0433]])

In [12]:
fig = dsf.plot(gif = True)
HTML(fig.to_html5_video())

In [ ]:
# Train continuous vector field
cvf = fields.vector_field.ContinuousVectorField()
cvf.train(dsf, epochs=100, nn=1, k=4, size=2000)

# Plot and save continuous vector field at center frame
fig = cvf.plot(dsf.coord_field, factor=12, frame=2)
fig.savefig(os.path.join(folder_name, "cvf_frame2.png"))
plt.close(fig)

# Extract learned parameters
sigma2 = cvf.sigma2
l0 = cvf.l0
l1 = cvf.l1
l2 = cvf.l2